In [2]:
import numpy as np
import pandas as pd

In [ ]:
#/Users/spencer.dork/Documents/NOAA Data/
#/Users/spencer.dork/Downloads/FCM Extraction Datasets/Biodiversity Data/Transcripts

### Biodiversity

In [205]:
dict_bio=pd.read_excel('/Users/spencer.dork/Documents/FCM/5-7 Original Biodiversity Data.xlsx', sheet_name=None, index_col=0)

In [66]:

row_labels = dict_bio['BD001'].index.tolist()
column_labels = dict_bio['BD001'].columns.tolist()

In [67]:
for sheet_name, df in dict_bio.items():
    dict_bio[sheet_name] = df.applymap(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))


/var/folders/3z/trrz86kd2jjf4smrjx86v6k80000gp/T/ipykernel_51956/923831242.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dict_bio[sheet_name] = df.applymap(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))


In [210]:
for fcm_group, matrix in dict_bio.items():
    # Rename row and column axis titles
    matrix.index.name = 'target'
    matrix.columns.name = 'source'

In [70]:
with pd.ExcelWriter('Biodiversity_FCM.xlsx') as writer:
    for sheet_name, df in dict_bio.items():
        df.to_excel(writer, sheet_name=sheet_name)


In [123]:
import os
import shutil

def group_files_by_prefix(base_folder, prefix_len=5):
    # Walk through all files in the directory and subdirectories
    for root, _, files in os.walk(base_folder):
        for file in files:
            # Skip hidden/system files
            if file.startswith('.'):
                continue

            # Get prefix and full paths
            prefix = file[:prefix_len]
            source_path = os.path.join(root, file)
            dest_folder = os.path.join(base_folder, prefix)

            # Create the destination folder if it doesn't exist
            os.makedirs(dest_folder, exist_ok=True)

            # Avoid moving into same folder repeatedly
            if os.path.abspath(root) != os.path.abspath(dest_folder):
                dest_path = os.path.join(dest_folder, file)

                # Move the file
                print(f"Moving: {source_path} -> {dest_path}")
                shutil.move(source_path, dest_path)

# === Example usage ===
target_folder = "/Users/spencer.dork/Documents/NOAA Data/Biodiversity/Transcripts"
group_files_by_prefix(target_folder)


Moving: /Users/spencer.dork/Documents/NOAA Data/Biodiversity/Transcripts/BD083.docx -> /Users/spencer.dork/Documents/NOAA Data/Biodiversity/Transcripts/BD083/BD083.docx
Moving: /Users/spencer.dork/Documents/NOAA Data/Biodiversity/Transcripts/BD017.docx -> /Users/spencer.dork/Documents/NOAA Data/Biodiversity/Transcripts/BD017/BD017.docx
Moving: /Users/spencer.dork/Documents/NOAA Data/Biodiversity/Transcripts/BD001.docx -> /Users/spencer.dork/Documents/NOAA Data/Biodiversity/Transcripts/BD001/BD001.docx
Moving: /Users/spencer.dork/Documents/NOAA Data/Biodiversity/Transcripts/BD076.docx -> /Users/spencer.dork/Documents/NOAA Data/Biodiversity/Transcripts/BD076/BD076.docx
Moving: /Users/spencer.dork/Documents/NOAA Data/Biodiversity/Transcripts/BD021.docx -> /Users/spencer.dork/Documents/NOAA Data/Biodiversity/Transcripts/BD021/BD021.docx
Moving: /Users/spencer.dork/Documents/NOAA Data/Biodiversity/Transcripts/BD037.docx -> /Users/spencer.dork/Documents/NOAA Data/Biodiversity/Transcripts/BD0

### Carobbean

In [72]:
dict_car=pd.read_excel('/Users/spencer.dork/Downloads/FCM Extraction Datasets/NOAA Caribbean Data/Caribbean Conceptual Models Dataset.xlsx', sheet_name=None)

In [73]:
fcm_mapping_df=dict_car['Sheet2']
cm_df=dict_car['Sheet 1']

In [74]:
fcm_to_columns = {}
for _, row in fcm_mapping_df.iterrows():
    if pd.notna(row['Column in CM dataset']):
        cols = [col.strip() for col in row['Column in CM dataset'].split(';') if col.strip() in cm_df.columns]
        fcm_to_columns[row['FCM ']] = cols

# === For each FCM group, find rows with at least one "1" in any of its columns ===
fcm_clumped_data = {}

for fcm_group, cols in fcm_to_columns.items():
    if not cols:
        continue
    # Filter rows where *any* of the group's columns has a 1
    mask = cm_df[cols].eq(1).any(axis=1)
    fcm_clumped_data[fcm_group] = cm_df[mask]

In [75]:
fcm_matrices = {}

for fcm_group, df in fcm_clumped_data.items():
    # Map '+' to 1, '-' to -1
    sign_map = df['Sign'].map({'+': 1, '-': -1})
    
    # Create a DataFrame with just Var1, Var2, and the mapped sign value
    temp = pd.DataFrame({
        'Var1': df['Var1'],
        'Var2': df['Var2'],
        'Value': sign_map
    })
    
    # Pivot to create matrix
    matrix = temp.pivot_table(index='Var2', columns='Var1', values='Value', aggfunc='first', fill_value=0)
    
    fcm_matrices[fcm_group] = matrix

In [76]:
for fcm_group, matrix in fcm_matrices.items():
    # Rename row and column axis titles
    matrix.index.name = 'target'
    matrix.columns.name = 'source'

In [78]:
with pd.ExcelWriter('NOAA_Caribbean_FCM.xlsx') as writer:
    for sheet_name, df in fcm_matrices.items():
        df.to_excel(writer, sheet_name=sheet_name)

In [86]:
import os
import shutil

def find_files_for_column(base_folder, column_name):
    """
    Search recursively for files/folders matching the column_name (case-insensitive).
    Returns list of full file paths found.
    """
    matches = []
    for root, dirs, files in os.walk(base_folder):
        # Search files
        for file in files:
            if column_name.lower() in file.lower():
                matches.append(os.path.join(root, file))
        # Optionally, also match folder names if needed:
        # for d in dirs:
        #     if column_name.lower() in d.lower():
        #         matches.append(os.path.join(root, d))
    return matches

def organize_files_by_fcm(base_folder, fcm_to_columns, output_folder):
    """
    For each FCM category, find all files corresponding to its columns and
    copy them into a new folder named after the FCM category under output_folder.
    """
    os.makedirs(output_folder, exist_ok=True)

    for fcm, columns in fcm_to_columns.items():
        fcm_folder = os.path.join(output_folder, fcm)
        os.makedirs(fcm_folder, exist_ok=True)

        print(f"Processing FCM group: {fcm}")

        for col in columns:
            print(f"  Searching for files related to: {col}")
            matched_files = find_files_for_column(base_folder, col)

            if not matched_files:
                print(f"    No files found for column: {col}")
                continue

            for filepath in matched_files:
                filename = os.path.basename(filepath)
                dest_path = os.path.join(fcm_folder, filename)
                
                # To avoid overwriting if duplicate names exist, you can add a prefix or suffix:
                if os.path.exists(dest_path):
                    name, ext = os.path.splitext(filename)
                    dest_path = os.path.join(fcm_folder, f"{name}_{os.path.basename(os.path.dirname(filepath))}{ext}")

                print(f"    Copying {filepath} to {dest_path}")
                shutil.copy2(filepath, dest_path)


In [121]:
fcm_to_columns = {}

for idx, row in fcm_mapping_df.iterrows():
    # Make sure to use the exact column names (watch spaces!)
    fcm_group = row['FCM ']  
    interviews_str = row['Interviews']

    if pd.notna(interviews_str):  # avoid NaNs
        fcm_to_columns[fcm_group] = [
            item.strip() for item in interviews_str.split(';') if item.strip()
        ]
    else:
        fcm_to_columns[fcm_group] = []
    

In [122]:
# === Example Usage ===
base_folder = "/Users/spencer.dork/Downloads/FCM Extraction Datasets/NOAA Caribbean Data"
output_folder = "/Users/spencer.dork/Documents/NOAA Data/Caribbean"

# fcm_to_columns should be your dict like:
# {
#   "PR Fishers": ["Fishers_Aguada PR", "Fishers_Anasco PR", ...],
#   "USVI Fishers": ["Fishers_STT/J", "Fishers_STX"],
#    ...
# }

organize_files_by_fcm(base_folder, fcm_to_columns, output_folder)

Processing FCM group: PR Fishers
  Searching for files related to: Key Informant - 8_PR
    Copying /Users/spencer.dork/Downloads/FCM Extraction Datasets/NOAA Caribbean Data/Key Informant Interviews PDFs/PR Key Informants/Key Informant - 8_PR.docx to /Users/spencer.dork/Documents/NOAA Data/Caribbean/PR Fishers/Key Informant - 8_PR.docx
  Searching for files related to: Key Informant - 9_PR
    Copying /Users/spencer.dork/Downloads/FCM Extraction Datasets/NOAA Caribbean Data/Key Informant Interviews PDFs/PR Key Informants/Key Informant - 9_PR.docx to /Users/spencer.dork/Documents/NOAA Data/Caribbean/PR Fishers/Key Informant - 9_PR.docx
  Searching for files related to: Key Informant - 12_PR
    Copying /Users/spencer.dork/Downloads/FCM Extraction Datasets/NOAA Caribbean Data/Key Informant Interviews PDFs/PR Key Informants/Key Informant - 12_PR.docx to /Users/spencer.dork/Documents/NOAA Data/Caribbean/PR Fishers/Key Informant - 12_PR.docx
  Searching for files related to: Key Informant -

In [124]:
fcm_to_columns

{'PR Fishers': ['Key Informant - 8_PR',
  'Key Informant - 9_PR',
  'Key Informant - 12_PR',
  'Key Informant - 16'],
 'USVI Fishers': ['Key Informant 3_USVI', 'Key Informant 4_USVI'],
 'PR Managers': ['Key Informant - 3_PR',
  'Key Informant - 6_PR',
  'Key Informant - 7_PR',
  'Key Informant - 10_PR',
  'Key Informant - 14'],
 'USVI Managers': ['Key Informant 10_USVI',
  'Key Informant 11_USVI',
  'Key Informant 12_USVI',
  'Key Informant 13_USVI',
  'Key Informant 14_USVI',
  'Key Informant 15_USVI',
  'Key Informant 17_USVI'],
 'PR Academics': ['Key Informant - 1_PR',
  'Key Informant - 2_PR',
  'Key Informant - 4_PR'],
 'USVI Academics': ['Key Informant 7_USVI',
  'Key Informant 18_USVI',
  'Key Informant 19_USVI',
  'Key Informant 20_USVI'],
 'PR NGOs': ['Key Informant - 5_PR',
  'Key Informant - 11_PR',
  'Key Informant - 13_PR'],
 'USVI NGOs': ['Key Informant 1_USVI',
  'Key Informant 2_USVI',
  'Key Informant 6_USVI'],
 'PR Businesses': ['Key Informant - 15'],
 'USVI Businesse

### Gulf OSW

In [39]:
import os

def create_folders_from_files(file_folder, output_folder):
    """
    For each file in file_folder, create a new folder in output_folder
    based on the file name (without extension).
    """
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    for file in os.listdir(file_folder):
        file_path = os.path.join(file_folder, file)

        if os.path.isfile(file_path):
            folder_name = os.path.splitext(file)[0]  # remove extension
            new_folder_path = os.path.join(output_folder, folder_name)

            # Create folder if it doesn't already exist
            if not os.path.exists(new_folder_path):
                os.makedirs(new_folder_path)
                print(f"Created: {new_folder_path}")
            else:
                print(f"Already exists: {new_folder_path}")

# === Example usage ===
source_folder = "/Users/spencer.dork/Downloads/FCM Extraction Datasets/Gulf OSW/FCM mmps/"
destination_folder = "/Users/spencer.dork/Documents/NOAA Data/Gulf OSW"
create_folders_from_files(source_folder, destination_folder)


Already exists: /Users/spencer.dork/Documents/NOAA Data/Gulf OSW/HRP
Already exists: /Users/spencer.dork/Documents/NOAA Data/Gulf OSW/DustyP
Already exists: /Users/spencer.dork/Documents/NOAA Data/Gulf OSW/SG
Already exists: /Users/spencer.dork/Documents/NOAA Data/Gulf OSW/SC
Already exists: /Users/spencer.dork/Documents/NOAA Data/Gulf OSW/KS
Already exists: /Users/spencer.dork/Documents/NOAA Data/Gulf OSW/JB
Already exists: /Users/spencer.dork/Documents/NOAA Data/Gulf OSW/JMF
Already exists: /Users/spencer.dork/Documents/NOAA Data/Gulf OSW/LP
Already exists: /Users/spencer.dork/Documents/NOAA Data/Gulf OSW/CH
Already exists: /Users/spencer.dork/Documents/NOAA Data/Gulf OSW/BZ
Already exists: /Users/spencer.dork/Documents/NOAA Data/Gulf OSW/MB
Already exists: /Users/spencer.dork/Documents/NOAA Data/Gulf OSW/UNO
Already exists: /Users/spencer.dork/Documents/NOAA Data/Gulf OSW/TRE
Already exists: /Users/spencer.dork/Documents/NOAA Data/Gulf OSW/AE
Already exists: /Users/spencer.dork/Docu

In [40]:
def load_as_square_matrix(csv_path):
    df = pd.read_csv(csv_path, index_col=0)

    # Drop unnamed columns that result from extra commas
    df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

    # Check if columns are unnamed/misaligned
    if df.columns.equals(pd.RangeIndex(len(df))):
        index_names = df.index.tolist()
        df.columns = index_names[:len(df.columns)]  # align to index length
    
    # df = df.fillna(0)

    # Make sure both axis names are strings to avoid Excel confusion
    df.columns.name = "target"
    df.index.name = "source"
    return df

def csvs_to_excel(input_folder, output_excel_path):
    data_dict = {}

    for file in os.listdir(input_folder):
        if file.endswith('.csv'):
            file_path = os.path.join(input_folder, file)
            sheet_name = os.path.splitext(file)[0][:31]

            # try:
            df = load_as_square_matrix(file_path)
            data_dict[sheet_name] = df.T
                # print(f"✅ Loaded sheet: {sheet_name}")
            # except Exception as e:
                # print(f"❌ Error processing {file}: {e}")

    return data_dict

In [41]:
input_folder = "Gulf OSW/"
output_excel_path = "/Users/spencer.dork/Documents/NOAA Data/Gulf OSW/Gulf_OSW_FCM.xlsx"

gulf_dict=csvs_to_excel(input_folder, output_excel_path)


In [42]:
gulf_dict['AE']

source,OSW Operation,OSW Construction,Reef Fish/Reef and Platform Ecology,Benthic Comm Health,Comm Reef Fisheries - Hook and Line,Rec Reef Fisheries - Hook and Line,Cabling,Rigs to Reefs,Human Navigation,Migratory Patterns,...,Acoustics,Telemetry/Tagging,change/familiarity,Willingness/Public Perception,Habitat Mapping - economic impacts of potential reef,Existing ONG,NaN,NaN,NaN,consortium
target,,,,,,,,,,,,,,,,,,,,,
OSW Operation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OSW Construction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Reef Fish/Reef and Platform Ecology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Benthic Comm Health,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Comm Reef Fisheries - Hook and Line,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rec Reef Fisheries - Hook and Line,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cabling,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rigs to Reefs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Human Navigation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:

def map_value(x):
    if isinstance(x, (int, float)):
        if x == 0:
            return 'Neutral'
        elif x > 0:
            return 1
        elif x<0:
            return -1
    return x

for fcm_group, matrix in gulf_dict.items():
    gulf_dict[fcm_group] = matrix.applymap(map_value)
    gulf_dict[fcm_group]=gulf_dict[fcm_group].fillna(0)
    # # Rename row and column axis titles
    # matrix.index.name = 'target'
    # matrix.columns.name = 'source'

/var/folders/3z/trrz86kd2jjf4smrjx86v6k80000gp/T/ipykernel_75097/1980867796.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  gulf_dict[fcm_group] = matrix.applymap(map_value)


In [45]:
pd.set_option('display.max_columns', None)

In [46]:
gulf_dict['AE']

source,OSW Operation,OSW Construction,Reef Fish/Reef and Platform Ecology,Benthic Comm Health,Comm Reef Fisheries - Hook and Line,Rec Reef Fisheries - Hook and Line,Cabling,Rigs to Reefs,Human Navigation,Migratory Patterns,Attraction/Reef Effect,Decommissioning,Ambient Temperature,Obstacles on bottom,bottom disturbances,Comm Fisheries - Trawl,Physical Footprint - pilings,NaN,workforce/income,ONG Economics - Jobs and Money,Obstacles in water column,"NW GoM Rig Reapers""""",MSP - distance between turbines,"Less Complexity"" comp to ONG""",Energy Generation,ROV Surveys,Data Baselines,Long term data/monitoring from beginning,Understanding/Ed to all stakeholders - combination of all producing,Forward thinking - End of Life,"Biodiversity""""",larval connectivity monitoring,eDNA,Acoustics,Telemetry/Tagging,change/familiarity,Willingness/Public Perception,Habitat Mapping - economic impacts of potential reef,Existing ONG,NaN,NaN,NaN,consortium
target,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
OSW Operation,0,0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0
OSW Construction,0,0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0
Reef Fish/Reef and Platform Ecology,0,0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0
Benthic Comm Health,0,0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,Neutral,-1.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,Neutral,Neutral,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0
Comm Reef Fisheries - Hook and Line,0,0,0.0,0.0,0.0,0.0,0,0,0.0,-1,1,0,Neutral,1,0.0,0.0,0,0.0,0,0.0,0,0.0,Neutral,0.0,0,0.0,Neutral,Neutral,Neutral,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0
Rec Reef Fisheries - Hook and Line,0,0,0.0,0.0,0.0,0.0,0,0,0.0,-1,1,0,Neutral,1,0.0,0.0,0,0.0,0,0.0,0,0.0,Neutral,0.0,0,0.0,Neutral,Neutral,Neutral,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0
Cabling,Neutral,Neutral,0.0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0
Rigs to Reefs,0,0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,Neutral,0,0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0
Human Navigation,0,0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,-1,0.0,0.0,0,0.0,0,0.0,0,0.0,Neutral,0.0,0,0.0,0,0,Neutral,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0


In [47]:
with pd.ExcelWriter('Gulf_OSW_FCM.xlsx') as writer:
    for sheet_name, df in gulf_dict.items():
        df.to_excel(writer, sheet_name=sheet_name)

### FLPP

In [96]:
FLPP_raw = pd.read_excel("/Users/spencer.dork/Downloads/All Maps.xlsx",sheet_name=None, index_col=0)

In [114]:
FLPP = {}

for sheet_name_str, df in FLPP_raw.items():
    # Convert sheet name to integer if possible
    try:
        sheet_key = int(sheet_name_str)
    except ValueError:
        sheet_key = sheet_name_str  # fallback if not a number

    # Convert index/columns to integers (if needed)
    df.index = df.index.map(lambda x: int(x) if isinstance(x, str) and str(x).isdigit() else x)
    df.columns = df.columns.map(lambda x: int(x) if isinstance(x, str) and str(x).isdigit() else x)

    # Fill NaNs with 0, then binarize to -1/0/1
    df = df.fillna(0).applymap(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))

    # Ensure consistent matrix structure
    df = df.sort_index().reindex(sorted(df.columns), axis=1)

    # Store in final dict with integer key
    FLPP[sheet_key] = df

/var/folders/3z/trrz86kd2jjf4smrjx86v6k80000gp/T/ipykernel_73966/1678903426.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.fillna(0).applymap(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
/var/folders/3z/trrz86kd2jjf4smrjx86v6k80000gp/T/ipykernel_73966/1678903426.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.fillna(0).applymap(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
/var/folders/3z/trrz86kd2jjf4smrjx86v6k80000gp/T/ipykernel_73966/1678903426.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.fillna(0).applymap(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
/var/folders/3z/trrz86kd2jjf4smrjx86v6k80000gp/T/ipykernel_73966/1678903426.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.fillna(0).applymap(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
/var/folders/3z/trrz86kd2jjf4smr

In [117]:
with pd.ExcelWriter('FLPP_FCM.xlsx') as writer:
    for sheet_name, df in FLPP.items():
        df.to_excel(writer, sheet_name=str(sheet_name))

In [118]:
import os
import shutil

def organize_docx_files(folder_path):
    # Ensure folder_path is absolute
    folder_path = os.path.abspath(folder_path)

    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.docx'):
            file_path = os.path.join(folder_path, filename)
            folder_name = os.path.splitext(filename)[0]
            new_folder_path = os.path.join(folder_path, folder_name)

            # Create folder if it doesn't exist
            os.makedirs(new_folder_path, exist_ok=True)

            # Destination path for the file
            dest_file_path = os.path.join(new_folder_path, filename)

            # Move the file into the new folder
            shutil.move(file_path, dest_file_path)
            print(f"Moved {filename} to {new_folder_path}")

# Example usage:
folder_with_docx = "/Users/spencer.dork/Documents/FCM Extraction Paper Data/FLPP/"
organize_docx_files(folder_with_docx)


Moved 108.docx to /Users/spencer.dork/Documents/FCM Extraction Paper Data/FLPP/108
Moved 149.docx to /Users/spencer.dork/Documents/FCM Extraction Paper Data/FLPP/149
Moved 132.docx to /Users/spencer.dork/Documents/FCM Extraction Paper Data/FLPP/132
Moved 124.docx to /Users/spencer.dork/Documents/FCM Extraction Paper Data/FLPP/124
Moved 153.docx to /Users/spencer.dork/Documents/FCM Extraction Paper Data/FLPP/153
Moved 104.docx to /Users/spencer.dork/Documents/FCM Extraction Paper Data/FLPP/104
Moved 112.docx to /Users/spencer.dork/Documents/FCM Extraction Paper Data/FLPP/112
Moved 145.docx to /Users/spencer.dork/Documents/FCM Extraction Paper Data/FLPP/145
Moved 128.docx to /Users/spencer.dork/Documents/FCM Extraction Paper Data/FLPP/128
Moved 129.docx to /Users/spencer.dork/Documents/FCM Extraction Paper Data/FLPP/129
Moved 144.docx to /Users/spencer.dork/Documents/FCM Extraction Paper Data/FLPP/144
Moved 113.docx to /Users/spencer.dork/Documents/FCM Extraction Paper Data/FLPP/113
Move